In [10]:
import numpy as np
# import scikit-allel
import allel
# check which version is installed
print(allel.__version__)

1.2.0


In [4]:
######### TAKES A LONG TIME #########
callset = allel.read_vcf('ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz')

In [5]:
sorted(callset.keys())

['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [12]:
np.where(callset['calldata/GT']!= 0)

(array([      0,       1,       1, ..., 1105537, 1105537, 1105537]),
 array([ 318,  356, 1689, ..., 1071, 1304, 2192]),
 array([1, 1, 1, ..., 0, 0, 0]))

In [13]:
callset["samples"]

array(['HG00096', 'HG00097', 'HG00099', ..., 'NA21142', 'NA21143',
       'NA21144'], dtype=object)

In [15]:
callset["variants/POS"]

array([ 9411239,  9411245,  9411264, ..., 48119697, 48119700, 48119740],
      dtype=int32)

In [16]:
gt = allel.GenotypeArray(callset['calldata/GT'])

<GenotypeArray shape=(1105538, 2504, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

In [17]:
callset["variants/ALT"]

array([['A', '', ''],
       ['A', '', ''],
       ['C', '', ''],
       ...,
       ['G', '', ''],
       ['G', '', ''],
       ['G', '', '']], dtype=object)

In [18]:
callset["variants/REF"]

array(['G', 'C', 'A', ..., 'T', 'A', 'C'], dtype=object)